In [2]:
import json
from pathlib import Path
import os

In [3]:
def integrate_annotations_into_unified_data(unified_data, cholect80_path):
    phase_annotations_path = Path(cholect80_path) / 'phase_annotations'
    
    for phase_file in phase_annotations_path.glob('*.txt'):
        video_id = "VID" + phase_file.stem.split('video')[1].split('-')[0].zfill(2)
        phases = parse_annotation_file(phase_file)

        for frame_id, phase in phases.items():
            if video_id not in unified_data:
                unified_data[video_id] = {"annotations": {}}
            if frame_id not in unified_data[video_id]["annotations"]:
                unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": [], "tools": []}
            
            unified_data[video_id]["annotations"][frame_id]["phase"] = phase


    return unified_data


In [4]:
def display_frame_annotations(frame_data):
    """
    Displays annotations for a specific frame, including phase information correctly.
    """
    print("    Categories:")
    for category, items in frame_data.get("categories", {}).items():
        items_str = ', '.join(str(item) for item in items if item is not None)
        print(f"      {category.capitalize()}: {items_str if items_str else 'N/A'}")
    
    # Display the phase directly
    phase = frame_data.get("phase", "N/A")
    print(f"    Phase: {phase}")
    

    # Continue with Q&A display as before
    if "qa" in frame_data:
        print("    Q&A:")
        for qa in frame_data["qa"]:
            print(f"      Question: {qa.get('question')}")
            for answer in qa.get("answers", []):
                print(f"        Answer: {answer}")
    print("")  # Newline for better readability


## Integrate CholecT50

In [5]:
json_file_path = '/media/telecom/77F7-D47D/unify_data/data.json'

cholect50_root_path = '/home/telecom/Downloads/CholecT50'
cholect50_labels_path = os.path.join(cholect50_root_path, 'labels')  
label_mapping_path = os.path.join(cholect50_root_path, 'label_mapping.txt')

label_mapping = {}
with open(label_mapping_path, 'r') as file:
    for line in file:
        parts = [part.strip() for part in line.strip().split(',')]
        
        try:
            triplet_id, instrument_id, verb_id, target_id, _, _ = [int(part) for part in parts]
            label_mapping[triplet_id] = {
                "instrument_id": instrument_id,
                "verb_id": verb_id,
                "target_id": target_id
            }
        except ValueError:
            print(f"Skipping line due to conversion issue: {line}")
            continue


def get_category_by_id(category_map, category_id):
    return category_map.get(category_id, "")

def build_categories_dict(categories):
    return {
        "instrument": {int(k): v for k, v in categories['instrument'].items()},
        "verb": {int(k): v for k, v in categories['verb'].items()},
        "target": {int(k): v for k, v in categories['target'].items()}
    }

def unify_cholect50_to_schema(cholect50_data, video_id, categories_dict):
    annotations = {}

    for frame_id, frame_annotations in cholect50_data['annotations'].items():
        frame_categories = {
            "instruments": [],
            "verbs": [],
            "targets": [],
        }

        for triplet in frame_annotations:
            triplet_id = triplet[0]
            triplet_mapping = label_mapping.get(triplet_id, {})

            instrument = get_category_by_id(categories_dict.get("instrument", {}), triplet_mapping.get("instrument_id", ""))
            verb = get_category_by_id(categories_dict.get("verb", {}), triplet_mapping.get("verb_id", ""))
            target = get_category_by_id(categories_dict.get("target", {}), triplet_mapping.get("target_id", ""))
            

            frame_categories["instruments"].append(instrument if instrument is not None else "")
            frame_categories["verbs"].append(verb if verb is not None else "")
            frame_categories["targets"].append(target if target is not None else "")

            # Extract bounding box coordinates
            bbox = {
                'x1': triplet[2],
                'y1': triplet[3],
                'width': triplet[4],
                'height': triplet[5]
            }

            frame_categories["instruments"].append(instrument)
            frame_categories["verbs"].append(verb)
            frame_categories["targets"].append(target)
            #frame_categories["phases"].append(phase_name)

            

        annotations[str(frame_id)] = {
            # ... specify bbox coordinates...
            "categories": frame_categories
        }
    
    video_data = {
        video_id: {
            "annotations": annotations
        }
    }
    
    return video_data

unified_data = {}

categories_dict = {}

for label_file in os.listdir(cholect50_labels_path):
    if label_file.endswith('.json'):
        label_file_path = os.path.join(cholect50_labels_path, label_file)
        
        with open(label_file_path, 'r') as file:
            cholect50_data = json.load(file)

        if not categories_dict:
            categories_dict = build_categories_dict(cholect50_data['categories'])

        video_id = label_file.replace('.json', '')
        
        video_data = unify_cholect50_to_schema(cholect50_data, video_id, categories_dict)
        unified_data.update(video_data)

with open(json_file_path, 'w') as outfile:
    json.dump(unified_data, outfile, indent=4)

print(f"Unified dataset has been saved to {json_file_path}")


Skipping line due to conversion issue: # IVT, I, V, T, IV, IT

Unified dataset has been saved to /media/telecom/77F7-D47D/unify_data/data.json


## Integrate SSG-VQA

In [6]:
def parse_qa_txt_file(qa_file_path):
    qa_data = []
    with open(qa_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            question, *answers = line.strip().split('|')
            if not question.endswith('?'):
                question += '?'
            qa_data.append({"question": question, "answers": [answer.strip() for answer in answers]})
    return qa_data

def integrate_qa_into_annotations(unified_data, video_id, frame_id, qa_data):
    if video_id not in unified_data:
        unified_data[video_id] = {"annotations": {}}
    if frame_id not in unified_data[video_id]["annotations"]:
        unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": []}

    # If 'qa' key doesn't exist within the frame, initialize it as an empty list
    frame_annotation = unified_data[video_id]["annotations"][frame_id]
    frame_annotation.setdefault("qa", [])

    frame_annotation["qa"].extend(qa_data)


def integrate_all_qa(sqa_path, unified_data):
    for video_dir in os.listdir(sqa_path):
        video_path = os.path.join(sqa_path, video_dir)
        if os.path.isdir(video_path):
            for qa_file in os.listdir(video_path):
                frame_id, _ = os.path.splitext(qa_file)
                qa_file_path = os.path.join(video_path, qa_file)
                qa_data = parse_qa_txt_file(qa_file_path)
                integrate_qa_into_annotations(unified_data, video_dir.upper(), frame_id, qa_data)

sqa_path = '/home/telecom/SSG-VQA/data/qa_txt' 
unified_json_file_path = '/media/telecom/77F7-D47D/unify_data/data.json' 

unified_data = {}
with open(unified_json_file_path, 'r') as file:
    unified_data = json.load(file)

integrate_all_qa(sqa_path, unified_data)

with open(unified_json_file_path, 'w') as file:
    json.dump(unified_data, file, indent=4)


## CholechT80

In [7]:
# Paths
cholect80_path = '/media/telecom/77F7-D47D/cholec80'  
unified_json_file_path = '/media/telecom/77F7-D47D/unify_data/data.json' 

In [8]:
def parse_annotation_file(file_path):
    """
    Parses a phase or tool annotation file and returns a dictionary keyed by frame ID.
    Each frame will have either a phase or a tool annotation.
    """
    annotations = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        next(file)  
        for line in file:
            parts = line.strip().split('\t')
            frame_id, annotation = parts[0], parts[1]
            annotations[frame_id] = annotation
    return annotations


In [9]:
def integrate_annotations_into_unified_data(unified_data, cholect80_path):
    phase_annotations_path = Path(cholect80_path) / 'phase_annotations'
    tool_annotations_path = Path(cholect80_path) / 'tool_annotations'
    
    for phase_file in phase_annotations_path.glob('*.txt'):
        video_id = "VID" + phase_file.stem.split('video')[1].split('-')[0].zfill(2)
        phases = parse_annotation_file(phase_file)

        for frame_id, phase in phases.items():
            if video_id not in unified_data:
                unified_data[video_id] = {"annotations": {}}
            if frame_id not in unified_data[video_id]["annotations"]:
                unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": [], "tools": []}
            unified_data[video_id]["annotations"][frame_id]["phase"] = phase
    
    for tool_file in tool_annotations_path.glob('*.txt'):
        video_id = "VID" + tool_file.stem.split('video')[1].split('-')[0].zfill(2)
        tools = parse_annotation_file(tool_file)

        for frame_id, tool in tools.items():
            if video_id not in unified_data:
                unified_data[video_id] = {"annotations": {}}
            if frame_id not in unified_data[video_id]["annotations"]:
                unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": [], "phase": ""}
            if "tools" not in unified_data[video_id]["annotations"][frame_id]:
                unified_data[video_id]["annotations"][frame_id]["tools"] = []
            unified_data[video_id]["annotations"][frame_id]["tools"].append(tool)

    return unified_data

In [10]:
with open(unified_json_file_path, 'r') as file:
    unified_data = json.load(file)

unified_data = integrate_annotations_into_unified_data(unified_data, cholect80_path)

with open(unified_json_file_path, 'w') as file:
    json.dump(unified_data, file, indent=4)

In [15]:
def parse_tool_annotations(file_path):
    tool_usage = {}
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        for row in reader:
            frame_id = row['Frame']
            present_tools = [tool for tool, presence in row.items() if tool != 'Frame' and presence == '1']
            tool_usage[frame_id] = present_tools
    return tool_usage

def integrate_tools_into_unified_data(unified_data, video_id, tool_usage):
    for frame_id, tools in tool_usage.items():
        if video_id not in unified_data:
            unified_data[video_id] = {"annotations": {}}
        if frame_id not in unified_data[video_id]["annotations"]:
            unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": []}
        
        unified_data[video_id]["annotations"][frame_id]["tools"] = tools

    return unified_data

### draft

In [16]:
def parse_phase_file(phase_file_path):
    """
    Parses a phase annotation file and returns a dictionary of phases keyed by frame ID.
    """
    phases = {}
    with open(phase_file_path, 'r', encoding='utf-8') as file:
        next(file)  # Skip the header line
        for line in file:
            parts = line.strip().split('\t')  # Assuming tab-separated values
            if len(parts) == 2:
                frame_id, phase = parts
                phases[frame_id] = phase
    return phases


def update_annotations(unified_data, video_id, phase_annotations, tool_annotations):
    """
    Updates the unified data with phase and tool annotations for a specific video.
    """
    if video_id not in unified_data:
        unified_data[video_id] = {"annotations": {}}

    for frame_id, phase in phase_annotations.items():
        if frame_id not in unified_data[video_id]["annotations"]:
            unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": [], "phases": [], "tools": []}
        unified_data[video_id]["annotations"][frame_id]["phases"].extend(phase)
    
    for frame_id, tools in tool_annotations.items():
        if frame_id not in unified_data[video_id]["annotations"]:
            unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": [], "phases": [], "tools": []}
        unified_data[video_id]["annotations"][frame_id]["tools"].extend(tools)

def integrate_cholect80(cholect80_path, unified_data):
    """
    Integrates CholecT80 data (both phase and tool annotations) into the unified dataset.
    """
    for video_num in range(1, 81):
        video_id = f"video{video_num:02d}"
        phase_file_path = Path(cholect80_path) / "phase_annotations" / f"{video_id}-phase.txt"
        tool_file_path = Path(cholect80_path) / "tool_annotations" / f"{video_id}-tool.txt"

        phase_annotations = parse_annotation_file(phase_file_path) if phase_file_path.exists() else {}
        tool_annotations = parse_annotation_file(tool_file_path) if tool_file_path.exists() else {}
        
        update_annotations(unified_data, video_id, phase_annotations, tool_annotations)

    return unified_data

def integrate_phases_into_unified_data(unified_data, video_id, phases):
    """
    Integrates phase annotations into the unified data for the specified video.
    """
    if video_id not in unified_data:
        unified_data[video_id] = {"annotations": {}}

    for frame_id, phase in phases.items():
        if frame_id not in unified_data[video_id]["annotations"]:
            unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": []}
        
        unified_data[video_id]["annotations"][frame_id]["phase"] = phase  

    return unified_data

with open(unified_json_file_path, 'r') as file:
    unified_data = json.load(file)

unified_data = integrate_cholect80(cholect80_path, unified_data)


phase_annotations_path = Path(cholect80_path) / 'phase_annotations'
for phase_file in phase_annotations_path.glob('*.txt'):
    video_id = phase_file.stem  
    phases = parse_phase_file(phase_file)
    unified_data = integrate_phases_into_unified_data(unified_data, video_id, phases)

with open(unified_json_file_path, 'w') as file:
    json.dump(unified_data, file, indent=4)

## Display with Vid ID and frame

In [17]:
def display_annotations(dataset, video_id, frame_id=None):
    """
    Display annotations for a specific video and optionally a specific frame within that video.
    """
    video_data = dataset.get(video_id, {})
    if not video_data:
        print(f"No data found for video ID: {video_id}")
        return

    annotations = video_data.get("annotations", {})
    if frame_id:
        frame_data = annotations.get(str(frame_id), {})
        if not frame_data:
            print(f"No data found for frame ID: {frame_id} in video ID: {video_id}")
            return
        print(f"Annotations for Video ID: {video_id}, Frame ID: {frame_id}:")
        display_frame_annotations(frame_data)
    else:
        for frame_id, frame_data in annotations.items():
            print(f"Annotations for Video ID: {video_id}, Frame ID: {frame_id}:")
            display_frame_annotations(frame_data)


def display_frame_annotations(frame_data):
    tool_names = ["Grasper", "Bipolar", "Hook", "Scissors", "Clipper", "Irrigator", "SpecimenBag"]
    
    print("    Categories:")
    for category, items in frame_data.get("categories", {}).items():
        items_str = ', '.join(str(item) for item in items if item is not None)
        print(f"      {category.capitalize()}: {items_str if items_str else 'N/A'}")
    
    phase = frame_data.get("phase", "N/A")
    print(f"    Phase: {phase}")

    tools_presence = frame_data.get("tools", [])
    if tools_presence:
        present_tools = [name for index, present in enumerate(tools_presence) if present == 1]
        if present_tools:
            print("    Tools:")
            print(f"      {', '.join(present_tools)}")
        else:
            print("    Tools: None")
    else:
        print("    Tools: N/A")

    if "qa" in frame_data:
        print("    Q&A:")
        for qa in frame_data["qa"]:
            print(f"      Question: {qa.get('question')}")
            answers = ', '.join(qa.get("answers", []))
            print(f"        Answers: {answers}")
    print("")  


unified_json_file_path = '/media/telecom/77F7-D47D/unify_data/data.json' 

with open(unified_json_file_path, 'r') as file:
    dataset = json.load(file)

video_id = "VID01"
frame_id = "0"  #  None for all frames 

display_annotations(dataset, video_id, frame_id)


Annotations for Video ID: VID01, Frame ID: 0:
    Categories:
      Instruments: grasper, grasper
      Verbs: grasp, grasp
      Targets: gallbladder, gallbladder
    Phase: Preparation
    Tools: None



In [38]:
tool_annotations_file = '/media/telecom/77F7-D47D/cholec80/tool_annotations/video01-tool.txt'  # Update with your actual file path
tool_usage = parse_tool_annotations(tool_annotations_file)

unified_data = integrate_tools_into_unified_data(unified_data, "VID01", tool_usage)

display_annotations(unified_data, "VID01", "1")  

Annotations for Video ID: VID01, Frame ID: 1:
    Categories:
      Instruments: grasper
      Verbs: grasp
      Targets: gallbladder
    Phase: Preparation
    Tools: N/A
    Triplets:
      (grasper, grasp, gallbladder)
    Q&A:
      Question: Which anatomical structures are present?
        Answer: abdominal_wall_cavity, liver, gut, omentum, gallbladder
      Question: Which tools are present?
        Answer: grasper
      Question: What is the grasper doing?
        Answer: grasp
      Question: Which tool is operating on gallbladder?
        Answer: grasper
      Question: What anatomy is at the top-left of the frame ?
        Answer: abdominal_wall_cavity
        Answer: [67.0, 85.5]
      Question: What anatomy is at the top-mid of the frame ?
        Answer: liver
        Answer: [212.0, 119.5]
      Question: What number of objects are both below the red top-left object and above the top-mid liver object?
        Answer: 0
        Answer: single_and.json
        Answer: coun

: 

## Analyse

In [20]:
def analyze_phase_transitions(unified_data, video_id):
    previous_phase = None
    transitions = []

    annotations = unified_data[video_id]["annotations"]
    for frame_id in sorted(annotations.keys(), key=lambda x: int(x)):
        current_phase = annotations[frame_id].get("phase")
        if current_phase != previous_phase:
            transitions.append((frame_id, current_phase))
            previous_phase = current_phase
    
    return transitions

# Example usage
transitions = analyze_phase_transitions(unified_data, "VID31")
print("Phase Transitions in VID31:")
for frame_id, phase in transitions:
    print(f"At frame {frame_id}, transitioned to '{phase}' phase.")


Phase Transitions in VID31:
At frame 0, transitioned to 'Preparation' phase.
At frame 4150, transitioned to 'CalotTriangleDissection' phase.
At frame 78075, transitioned to 'ClippingCutting' phase.
At frame 89075, transitioned to 'GallbladderDissection' phase.
At frame 94675, transitioned to 'GallbladderPackaging' phase.
At frame 97150, transitioned to 'CleaningCoagulation' phase.
At frame 98525, transitioned to 'GallbladderRetraction' phase.


In [21]:
def analyze_all_phase_transitions(unified_data):
    all_transitions = {}
    l= []
    for video_id in unified_data.keys():
        previous_phase = None
        transitions = []

        annotations = unified_data[video_id]["annotations"]
        for frame_id in sorted(annotations.keys(), key=lambda x: int(x)):
            current_phase = annotations[frame_id].get("phase")
            if current_phase != previous_phase:
                transitions.append((frame_id, current_phase))
                previous_phase = current_phase
        
        l.append(frame_id)
        
        all_transitions[video_id] = transitions
    print(l)
    return all_transitions 

# Example usage
all_transitions = analyze_all_phase_transitions(unified_data)

for video_id, transitions in all_transitions.items():
    print(f"Phase Transitions in {video_id}:")
    for frame_id, phase in transitions:
        print(f"  At frame {frame_id}, transitioned to '{phase}' phase.")
    print("\n")  

['98625', '2219', '63300', '2123', '40875', '18475', '38050', '65800', '52650', '42700', '46825', '33900', '70975', '55550', '45850', '2176', '58600', '40850', '59050', '43325', '29850', '59675', '45600', '52100', '27350', '51451', '2145', '58750', '92800', '1706', '43100', '56475', '41775', '45850', '43725', '44325', '53825', '24525', '52900', '73600', '53225', '37975', '27250', '49300', '49150', '85350', '48075', '50800', '48550', '38300', '2358', '2532', '1531', '2532', '2838', '1531', '1634', '1634', '2360', '2839', '145700', '113925', '67550', '80500', '77575', '73925', '32600', '60600', '36225', '78175', '84675', '41325', '31450', '49375', '29975', '73125', '32675', '33075', '30801', '77000', '82075', '77500', '41175', '25925', '149825', '26150', '110225', '85825', '59950', '58825', '114375', '62875', '77650', '66225', '62550', '43325', '70975', '145700', '38050', '58600', '53825', '113925', '37975', '67550', '43725', '80500', '27250', '24525', '42700', '51451', '73925', '32600',

In [32]:
def extract_actions_for_phase(unified_data, video_id, phase):
    actions = []

    annotations = unified_data[video_id]["annotations"]
    for frame_id, data in annotations.items():
        if data.get("phase") == phase:
            categories = data.get("categories", {})
            for i in range(len(categories.get("instruments", []))):
                action = (
                    categories["instruments"][i],
                    categories["verbs"][i],
                    categories["targets"][i]
                )
                actions.append(action)
    
    return set(actions)  # Using a set to eliminate duplicates

# Example usage
preparation_actions = extract_actions_for_phase(unified_data, "VID01", "Preparation")
print("Actions during the Preparation phase in VID01:")
for action in preparation_actions:
    print(action)

IndexError: list index out of range

In [27]:
def get_frame_annotations(unified_data, video_id, frame_id):
    video_data = unified_data.get(video_id, {})
    return video_data.get("annotations", {}).get(frame_id, {})

# Example usage
frame_annotations = get_frame_annotations(unified_data, "VID01", "1")
print(f"Annotations for VID01, frame 1: {frame_annotations}")


Annotations for VID01, frame 1: {'categories': {'instruments': ['grasper'], 'verbs': ['grasp'], 'targets': ['gallbladder']}, 'qa': [{'question': 'Which anatomical structures are present?', 'answers': ['abdominal_wall_cavity, liver, gut, omentum, gallbladder']}, {'question': 'Which tools are present?', 'answers': ['grasper']}, {'question': 'What is the grasper doing?', 'answers': ['grasp']}, {'question': 'Which tool is operating on gallbladder?', 'answers': ['grasper']}, {'question': 'What anatomy is at the top-left of the frame ?', 'answers': ['abdominal_wall_cavity', '[67.0, 85.5]']}, {'question': 'What anatomy is at the top-mid of the frame ?', 'answers': ['liver', '[212.0, 119.5]']}, {'question': 'What number of objects are both below the red top-left object and above the top-mid liver object?', 'answers': ['0', 'single_and.json', 'count', 'NA']}, {'question': 'How many anatomys are both right of the abdominal_wall_cavity thing and to the left of the top-mid liver thing?', 'answers'

## Compare tools

In [28]:
import csv

def parse_tool_annotations(file_path):
    tool_usage = {}
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        for row in reader:
            frame_id = row['Frame']
            present_tools = [tool for tool, presence in row.items() if tool != 'Frame' and presence == '1']
            tool_usage[frame_id] = present_tools
    return tool_usage


In [29]:
def integrate_tools_into_unified_data(unified_data, video_id, tool_usage):
    for frame_id, tools in tool_usage.items():
        if video_id not in unified_data:
            unified_data[video_id] = {"annotations": {}}
        if frame_id not in unified_data[video_id]["annotations"]:
            unified_data[video_id]["annotations"][frame_id] = {"categories": {}, "qa": []}
        
        unified_data[video_id]["annotations"][frame_id]["tools"] = tools

    return unified_data


In [30]:
def display_frame_annotations(frame_data):
    print("    Categories:")
    for category, items in frame_data.get("categories", {}).items():
        items_str = ', '.join(str(item) for item in items if item is not None)
        print(f"      {category.capitalize()}: {items_str if items_str else 'N/A'}")
    
    phase = frame_data.get("phase", "N/A")
    print(f"    Phase: {phase}")

    if "tools" in frame_data:
        tools_str = ', '.join(frame_data["tools"])
        print("    Tools:")
        print(f"      {tools_str if tools_str else 'None'}")
    else:
        print("    Tools: N/A")

    # Display triplets if available
    if "triplets" in frame_data:
        print("    Triplets:")
        for triplet in frame_data["triplets"]:
            print(f"      {triplet}")
    else:
        print("    Triplets: N/A")

    if "qa" in frame_data:
        print("    Q&A:")
        for qa in frame_data["qa"]:
            print(f"      Question: {qa.get('question')}")
            for answer in qa.get("answers", []):
                print(f"        Answer: {answer}")
    print("")  # Adds an empty line for readability


In [37]:
tool_annotations_file = '/media/telecom/77F7-D47D/cholec80/tool_annotations/video01-tool.txt' 
tool_usage = parse_tool_annotations(tool_annotations_file)

unified_data = integrate_tools_into_unified_data(unified_data, "VID01", tool_usage)

display_annotations(unified_data, "VID01", "1100")
#print(type(unified_data))
#sorted(unified_data.keys())

Annotations for Video ID: VID01, Frame ID: 1100:
    Categories:
      Instruments: hook, grasper
      Verbs: dissect, retract
      Targets: gallbladder
    Phase: CalotTriangleDissection
    Tools:
      Grasper, Hook
    Triplets:
      (hook, dissect, gallbladder)



In [33]:
def generate_triplets_from_frame(frame_data):
    instruments = frame_data.get("categories", {}).get("instruments", [])
    verbs = frame_data.get("categories", {}).get("verbs", [])
    targets = frame_data.get("categories", {}).get("targets", [])
    
    # Ensure there's an equal number of instruments, verbs, and targets
    # This might need adjustment based on how your data is structured
    triplets = list(zip(instruments, verbs, targets))
    return ["(" + ", ".join(triplet) + ")" for triplet in triplets]

In [34]:
def integrate_triplets_into_unified_data(unified_data):
    for video_id, video_data in unified_data.items():
        for frame_id, frame_data in video_data.get("annotations", {}).items():
            triplets = generate_triplets_from_frame(frame_data)
            frame_data["triplets"] = triplets
    return unified_data

In [35]:
unified_data = integrate_triplets_into_unified_data(unified_data)

In [36]:
with open(unified_json_file_path, 'w', encoding='utf-8') as file:
    json.dump(unified_data, file, indent=4)

## Remove Duplicates

In [18]:
def remove_repetitions(unified_data):
    for video_id, video_data in unified_data.items():
        annotations = video_data.get("annotations", {})
        for frame_id, frame_data in annotations.items():
            categories = frame_data.get("categories", {})
            # Remove repetitions for instruments, verbs, and targets
            for category, items in categories.items():
                unique_items = list(set(items))  # Convert to set to remove duplicates, then back to list
                unified_data[video_id]["annotations"][frame_id]["categories"][category] = unique_items
            
            # Additionally, remove repetitions in triplets
            if "triplets" in frame_data:
                # Assuming triplets are strings and can be directly deduplicated using a set
                unique_triplets = list(set(frame_data["triplets"]))
                unified_data[video_id]["annotations"][frame_id]["triplets"] = unique_triplets

    return unified_data

# Assuming unified_data is your dataset loaded into a Python dictionary
# Apply the function to remove repetitions
unified_data = remove_repetitions(unified_data)

# Optionally, save the updated dataset back to a JSON file
with open(unified_json_file_path, 'w', encoding='utf-8') as file:
    json.dump(unified_data, file, indent=4)

print("Repetitions removed and dataset updated.")

Repetitions removed and dataset updated.
